<div style="background-color:#004D40; padding:20px; border-radius:12px;">
  <h1 style="color:#FFEB3B; text-align:center; font-size:30px; margin:0;">
    Application prédictive – Détection des faux billets
  </h1>
</div>


In [2]:
# Import des bibliothèques
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

In [3]:
# Chargement du modèle SVM et du scaler
modele = joblib.load("modele_svm.pkl")
scaler = joblib.load("scaler.pkl")


In [4]:
# Chargement du fichier billets_production.csv 
df_prod = pd.read_csv("billets_production.csv", sep=";")
df_prod.head()


diagonal  height_left  height_right  margin_low  margin_up  length   id
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5

In [5]:
# Séparation des variables numériques (pour standardisation)
X_prod = df_prod.drop(columns="id")

In [6]:
# Standardisation des données de production
X_prod_scaled = scaler.transform(X_prod)

In [7]:
# Prédiction avec le modèle SVM
predictions = modele.predict(X_prod_scaled)

In [8]:
# Ajout des résultats au DataFrame original
df_prod["prediction"] = predictions


In [9]:
# Obtenir les probabilités pour chaque billet (classe 0 et 1)
probas = modele.predict_proba(X_prod_scaled)

# Ajouter les deux colonnes de probabilité (classe 0 et 1)
df_prod["proba_faux"] = probas[:, 0]  # proba que le billet soit faux
df_prod["proba_vrai"] = probas[:, 1]  # proba que le billet soit vrai


In [10]:
df_prod

diagonal  height_left  height_right  margin_low  margin_up  length   id  \
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1   
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2   
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3   
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4   
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5   

   prediction  proba_faux  proba_vrai  
0           0    0.998421    0.001579  
1           0    0.999822    0.000178  
2           0    0.999820    0.000180  
3           1    0.012034    0.987966  
4           1    0.000001    0.999999

In [11]:
# remplacer des valeurs numériques par des libellés
df_prod["prediction"] = df_prod["prediction"].map({0: "Faux billet", 1: "Vrai billet"})
df_prod

diagonal  height_left  height_right  margin_low  margin_up  length   id  \
0    171.76       104.01        103.54        5.21       3.30  111.42  A_1   
1    171.87       104.17        104.13        6.00       3.31  112.09  A_2   
2    172.00       104.58        104.29        4.99       3.39  111.57  A_3   
3    172.49       104.55        104.34        4.44       3.03  113.20  A_4   
4    171.65       103.63        103.56        3.77       3.16  113.33  A_5   

    prediction  proba_faux  proba_vrai  
0  Faux billet    0.998421    0.001579  
1  Faux billet    0.999822    0.000178  
2  Faux billet    0.999820    0.000180  
3  Vrai billet    0.012034    0.987966  
4  Vrai billet    0.000001    0.999999